In [1]:
import os
import javalang
import pickle
from gensim.models.word2vec import Word2Vec
import numpy as np
import re
import string
import random
import sys


In [2]:
def solve_camel_and_underline(token):
    if token.isdigit():
        return [token]
    else:
        p = re.compile(r'([a-z]|\d)([A-Z])')
        sub = re.sub(p, r'\1_\2', token).lower()
        sub_tokens = sub.split("_")
        tokens = re.sub(" +", " ", " ".join(sub_tokens)).strip()
        final_token = []
        for factor in tokens.split(" "):
            final_token.append(factor.rstrip(string.digits))
        return final_token

In [3]:
def cut_data(token_seq, token_length_for_reserve):
    if len(token_seq) <= token_length_for_reserve:
        return token_seq
    else:
        start_index = token_seq.index("rank2fixstart")
        end_index = token_seq.index("rank2fixend")
        assert end_index > start_index
        length_of_annotated_statement = end_index - start_index + 1
        if length_of_annotated_statement <= token_length_for_reserve:
            padding_length = token_length_for_reserve - length_of_annotated_statement
            # give 2/3 padding space to content before annotated statement
            pre_padding_length = padding_length // 3 * 2
            # give 1/3 padding space to content after annotated statement
            post_padding_length = padding_length - pre_padding_length
            if start_index >= pre_padding_length and len(token_seq) - end_index - 1 >= post_padding_length:
                return token_seq[start_index - pre_padding_length: end_index + 1 + post_padding_length]
            elif start_index < pre_padding_length:
                return token_seq[:token_length_for_reserve]
            elif len(token_seq) - end_index - 1 < post_padding_length:
                return token_seq[len(token_seq) - token_length_for_reserve:]
        else:
            return token_seq[start_index: start_index + token_length_for_reserve]

In [14]:
def generate_token_seq(tokens,token_length_for_reserve):
	last_token=None
	t_seq = []
	for token in tokens:
		if isinstance(token, javalang.tokenizer.String):
			tmp_token = ["stringliteral"]
		else:
			tmp_token = solve_camel_and_underline(token.value)
			if last_token is not None and isinstance(token, javalang.tokenizer.Identifier) and (
					isinstance(last_token, javalang.tokenizer.Identifier) or isinstance(last_token,
																						   javalang.tokenizer.BasicType)) and last_token.value!='rank2fixstart':
				cur_token=tmp_token
				tmp_token=[]
				type_token_seq = solve_camel_and_underline(last_token.value)
				for t_token in cur_token:
					tmp_token.append([type_token_seq, cur_token[:cur_token.index(t_token)], t_token])
		last_token=token
		t_seq += tmp_token
	return cut_data(t_seq, token_length_for_reserve)

In [12]:
def token2num(tokens,vocab_dict,index_oov):
    num_seq=[]
    for token in tokens:
        if type(token)==str:
            num=vocab_dict[token] if token in vocab_dict else index_oov
        else:

            identifiers=token[0]
            prev_token=token[1]
            cur_token=token[2]
            iden_nums=[vocab_dict[t] if token in vocab_dict else index_oov for t in identifiers]
            prev_nums=[vocab_dict[t] if token in vocab_dict else index_oov for t in prev_token]
            cur_num=vocab_dict[cur_token] if token in vocab_dict else index_oov
            num=[iden_nums,prev_nums,cur_num]
        num_seq.append(num)
    return num_seq

In [6]:
# Parameters declaration
print("Parameters declaration")
tags = ["positive", "negative"]
train_prop = 0.8
val_prop = 0.1
test_prop = 0.1
shuffle_seed = 666
vector_size = 32
token_length_for_reserve = 400
extend_size = 2
max_vocab_size = 50000

Parameters declaration


In [7]:
current_pattern='InsertMissedStmt'
print("Path declaration")
fl_dataset_path = "../../dataset_fl.pkl"
output_data_dir = "../data/{}/".format(current_pattern)
if not os.path.exists(output_data_dir):
    os.makedirs(output_data_dir)

Path declaration


In [8]:
# split train/val/test parts and generate training corpus for word2vec pre-training
with open(fl_dataset_path, "rb") as file:
    dataset_fl = pickle.load(file)

In [23]:
one_method=dataset_fl['positive'][current_pattern][10]
print(one_method)

@Override public void onError(Throwable t){
  if (done) {
 rank2fixstart     return; rank2fixend 
  }
  done=true;
  if (s == null) {
    CompositeException t2=new CompositeException(t,new NullPointerException("Subscription not set!"));
    try {
      actual.onSubscribe(EmptyDisposable.INSTANCE);
    }
 catch (    Throwable e) {
      Exceptions.throwIfFatal(e);
      t2.suppress(e);
      RxJavaPlugins.onError(t2);
      return;
    }
    try {
      actual.onError(t2);
    }
 catch (    Throwable e) {
      Exceptions.throwIfFatal(e);
      t2.suppress(e);
      RxJavaPlugins.onError(t2);
    }
    return;
  }
  if (t == null) {
    t=new NullPointerException();
  }
  try {
    actual.onError(t);
  }
 catch (  Throwable ex) {
    Exceptions.throwIfFatal(ex);
    RxJavaPlugins.onError(new CompositeException(t,ex));
  }
}


In [24]:
method=one_method.strip()
tokens=javalang.tokenizer.tokenize(method)
token_seq=generate_token_seq(tokens,token_length_for_reserve)

In [25]:
print(token_seq)

['@', 'override', 'public', 'void', 'on', 'error', '(', 'throwable', [['throwable'], [], 't'], ')', '{', 'if', '(', 'done', ')', '{', 'rank2fixstart', 'return', ';', 'rank2fixend', '}', 'done', '=', 'true', ';', 'if', '(', 's', '==', 'null', ')', '{', 'composite', 'exception', [['composite', 'exception'], [], 't'], '=', 'new', 'composite', 'exception', '(', 't', ',', 'new', 'null', 'pointer', 'exception', '(', 'stringliteral', ')', ')', ';', 'try', '{', 'actual', '.', 'on', 'subscribe', '(', 'empty', 'disposable', '.', 'instance', ')', ';', '}', 'catch', '(', 'throwable', [['throwable'], [], 'e'], ')', '{', 'exceptions', '.', 'throw', 'if', 'fatal', '(', 'e', ')', ';', 't', '.', 'suppress', '(', 'e', ')', ';', 'rx', 'java', 'plugins', '.', 'on', 'error', '(', 't', ')', ';', 'return', ';', '}', 'try', '{', 'actual', '.', 'on', 'error', '(', 't', ')', ';', '}', 'catch', '(', 'throwable', [['throwable'], [], 'e'], ')', '{', 'exceptions', '.', 'throw', 'if', 'fatal', '(', 'e', ')', ';', 't

In [26]:
print("load vocab and vectors")
with open(os.path.join(output_data_dir, "vocab.pkl"), "rb") as file:
    vocab_dict=pickle.load(file)
with open(os.path.join(output_data_dir, "vectors.pkl"), "rb") as file:
    vectors= pickle.load(file)

load vocab and vectors


In [27]:
print("Generating train/val/test dataset")
index_oov = vocab_dict["OOV"]
index_padding = vocab_dict["PADDING"]
current_index = len(vocab_dict)

Generating train/val/test dataset


In [ ]:
normal_record = token2num(token_seq,vocab_dict,index_oov)
if len(normal_record) < token_length_for_reserve:
    normal_record += [index_padding] * (token_length_for_reserve - len(normal_record))


In [ ]:
token_seq_dataset = {"train": {}, "val": {}, "test": {}}
w2v_training_corpus = []
for tag in dataset_fl:
    for part in token_seq_dataset:
        token_seq_dataset[part][tag] = []
    random_copy = dataset_fl[tag][current_pattern][:]
    random.seed(shuffle_seed)
    random.shuffle(random_copy)
    for index, method in enumerate(random_copy):
        method = method.strip()
        tokens = javalang.tokenizer.tokenize(method)

        token_seq = generate_token_seq(tokens,token_length_for_reserve)

        if index + 1 < len(random_copy) * train_prop:
            token_seq_dataset["train"][tag].append(token_seq)
        elif index + 1 < len(random_copy) * (train_prop + val_prop):
            token_seq_dataset["val"][tag].append(token_seq)
        else:
            token_seq_dataset["test"][tag].append(token_seq)